In [1]:
import os
import torch
import wandb
import numpy as np
import pandas as pd
import warnings
import random
import re

from accelerate import infer_auto_device_map
from peft import prepare_model_for_kbit_training

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    TrainerCallback,
    EarlyStoppingCallback,
    TrainerState,
    TrainerControl,
)
from datasets import load_dataset, Dataset
from trl import SFTTrainer, setup_chat_format
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import bitsandbytes as bnb

In [2]:
# huggingface-cli login --token key   
# wandb login --relogin key

In [3]:
warnings.filterwarnings("ignore")
#https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction
# Load the application_record dataset
data = pd.read_csv("/opt/notebooks/Chatbot-Credit-Card/backend/dataset/credit-card-approval/application_record.csv")
# Load the credit_record dataset
record = pd.read_csv("/opt/notebooks/Chatbot-Credit-Card/backend/dataset/credit-card-approval/credit_record.csv")
# Find the first account open month for each user
begin_month = record.loc[record.groupby("ID")["MONTHS_BALANCE"].idxmin()]
begin_month = begin_month.rename(columns={"MONTHS_BALANCE": "begin_month"})

# Merge the datasets
df = pd.merge(data, begin_month, how="left", on="ID")
print("Datasets loaded and merged successfully.")
# Define approval logic based on multiple criteria
def determine_approval(row):
    # Define custom approval logic
    if row["STATUS"] in ["0", "1", "C", "X"]:  # Good credit status
            return 1  # Approved
    return 0  # Default to denial if STATUS is bad or missing

# Apply logic to determine approval (filling missing STATUS values first)
record["STATUS"] = record["STATUS"].fillna("X")  # Handle missing values
record["Approved"] = record.apply(determine_approval, axis=1)
# Aggregate approval status for each ID (disapproval if any ID has disqualifying criteria)
approval_status = record.groupby("ID")["Approved"].min().reset_index()
# Merge approval status back into the main dataset, avoiding "_x" and "_y" columns
df = pd.merge(data, approval_status, how="left", on="ID")
df["Approved"] = df["Approved"].fillna(0).astype(int)  # Fill missing approvals as denial
print("Approval status merged successfully.")
# Preprocess the 'DAYS_BIRTH' column to convert days to years
df['DAYS_BIRTH'] = (-df['DAYS_BIRTH'] // 365).fillna(0).astype(int)
df.drop(columns=['ID'], inplace=True)
# Preprocess the 'DAYS_EMPLOYED' column to get absolute values and handle unemployment
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].apply(lambda x: abs(x) if x < 0 else 0)
# Handle missing or infinite values in numerical columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numerical_cols] = df[numerical_cols].replace([np.inf, -np.inf], np.nan)  # Replace infinities with NaN
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())  # Fill NaN with median values
print("Preprocessing completed successfully.")

# Define the feature mapping dictionary
feature_mapping = {
    'CODE_GENDER': 'Gender',
    'FLAG_OWN_CAR': 'Car Ownership',
    'FLAG_OWN_REALTY': 'Property Ownership',
    'CNT_CHILDREN': 'Number of Children',
    'AMT_INCOME_TOTAL': 'Annual Income',
    'NAME_INCOME_TYPE': 'Income Category',
    'NAME_EDUCATION_TYPE': 'Education Level',
    'NAME_FAMILY_STATUS': 'Marital Status',
    'NAME_HOUSING_TYPE': 'Housing Type',
    'DAYS_BIRTH': 'Age (Days)',
    'DAYS_EMPLOYED': 'Employment Duration (Days)',
    'FLAG_MOBIL': 'Mobile Phone',
    'FLAG_WORK_PHONE': 'Work Phone',
    'FLAG_PHONE': 'Phone',
    'FLAG_EMAIL': 'Email',
    'OCCUPATION_TYPE': 'Occupation',
    'CNT_FAM_MEMBERS': 'Family Size',
    'STATUS': 'Credit Status'
}

# Rename the columns in the DataFrame using the mapping
df.rename(columns=feature_mapping, inplace=True)

# Display the first few rows to confirm the changes
df.head()
df.rename(columns={'TARGET': 'Approved'}, inplace=True)
# Display the first few rows to confirm the change
df.head()

Datasets loaded and merged successfully.
Approval status merged successfully.
Preprocessing completed successfully.


,Gender,Car Ownership,Property Ownership,Number of Children,Annual Income,Income Category,Education Level,Marital Status,Housing Type,Age (Days),Employment Duration (Days),Mobile Phone,Work Phone,Phone,Email,Occupation,Family Size,Approved
0,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32,4542.0,1.0,1.0,0.0,0.0,NaN,2.0,1
1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32,4542.0,1.0,1.0,0.0,0.0,NaN,2.0,1
2,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,58,1134.0,1.0,0.0,0.0,0.0,Security staff,2.0,1
3,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52,3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,1
4,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52,3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,1


In [4]:
explanation_df = pd.read_csv('/opt/notebooks/Chatbot-Credit-Card/backend/dataset/explanations_df.csv')
# explanation_df.drop(['Approved','Explanation'], axis=1, inplace=True)
explanation_df.head()

,Prediction,Actual,Explanation
0,Approved,Approved,This application was approved due to Employmen...
1,Approved,Approved,This application was approved due to Employmen...
2,Approved,Approved,"This application was approved due to -0.77, Ma..."
3,Approved,Approved,"This application was approved due to Email, -0..."
4,Denied,Approved,"This application was denied due to -1.62, Emai..."


In [5]:
print(explanation_df['Explanation'].iloc[0])

This application was approved due to Employment Duration (Days), Housing Type, -0.77, Marital Status, Work Phone, -0.15, Number of Children, -0.81, Family Size, Annual Income.


In [6]:
# Function to remove numbers and clean the Explanation column
def clean_explanation(text):
    return re.sub(r'[-+]?\d*\.\d+|\d+', '', text).replace(', ,', ',').replace(' ,', ',').strip(", ")

# Apply the cleaning function to the Explanation column
explanation_df["Explanation"] = explanation_df["Explanation"].apply(clean_explanation)

In [7]:
# Display the DataFrame with the cleaned Explanation column
print(explanation_df['Explanation'].iloc[0])
explanation_df.head()

This application was approved due to Employment Duration (Days), Housing Type, Marital Status, Work Phone, Number of Children, Family Size, Annual Income.


,Prediction,Actual,Explanation
0,Approved,Approved,This application was approved due to Employmen...
1,Approved,Approved,This application was approved due to Employmen...
2,Approved,Approved,"This application was approved due to, Marital ..."
3,Approved,Approved,"This application was approved due to Email, Nu..."
4,Denied,Approved,"This application was denied due to, Email, Hou..."


In [8]:
# df.rename(columns={'Age (Days)': 'Age (Years)'}, inplace=True)
df.rename(columns={'Employment Duration' : 'Employment Duration (Days)'}, inplace=True)
df['Age (Days)'] = df['Age (Days)'].astype(str) + " years old"

In [9]:
# Combine explanation_df and df based on index
df['Reason'] = explanation_df['Explanation'].reset_index(drop=True)

# Display the first value in the 'Reason' column
print(df['Reason'].iloc[0])
df.head()

This application was approved due to Employment Duration (Days), Housing Type, Marital Status, Work Phone, Number of Children, Family Size, Annual Income.


,Gender,Car Ownership,Property Ownership,Number of Children,Annual Income,Income Category,Education Level,Marital Status,Housing Type,Age (Days),Employment Duration (Days),Mobile Phone,Work Phone,Phone,Email,Occupation,Family Size,Approved,Reason
0,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32 years old,4542.0,1.0,1.0,0.0,0.0,NaN,2.0,1,This application was approved due to Employmen...
1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32 years old,4542.0,1.0,1.0,0.0,0.0,NaN,2.0,1,This application was approved due to Employmen...
2,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,58 years old,1134.0,1.0,0.0,0.0,0.0,Security staff,2.0,1,"This application was approved due to, Marital ..."
3,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52 years old,3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,1,"This application was approved due to Email, Nu..."
4,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52 years old,3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,1,"This application was denied due to, Email, Hou..."


In [10]:
# Function to dynamically replace labels with row values
def insert_numbers_dynamically(row):
    reason = row['Reason']
    
    # Extract all parts of the reason where dynamic replacement is needed
    labels = re.findall(r'([A-Za-z\s()]+)', reason)
    
    for label in labels:
        # Match the label to the corresponding column name
        column_name = None
        for col in df.columns:
            # Normalize column names and labels for matching
            normalized_col = re.sub(r'[\s()]+', '', col).lower()
            normalized_label = re.sub(r'[\s()]+', '', label).lower()
            
            if normalized_col == normalized_label:
                column_name = col
                break
        
        if column_name and column_name in row:  # Ensure column exists in the DataFrame
            # Replace the label with the corresponding value from the row
            value = row[column_name]
            # Ensure proper replacement in the text
            reason = reason.replace(label, f"{label.strip()} {value}")
    
    return reason

In [11]:
# Apply the function to each row in the DataFrame
df['Reason'] = df.apply(insert_numbers_dynamically, axis=1)

# Display the updated DataFrame
df.head()

,Gender,Car Ownership,Property Ownership,Number of Children,Annual Income,Income Category,Education Level,Marital Status,Housing Type,Age (Days),Employment Duration (Days),Mobile Phone,Work Phone,Phone,Email,Occupation,Family Size,Approved,Reason
0,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32 years old,4542.0,1.0,1.0,0.0,0.0,NaN,2.0,1,This application was approved due to Employmen...
1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32 years old,4542.0,1.0,1.0,0.0,0.0,NaN,2.0,1,This application was approved due to Employmen...
2,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,58 years old,1134.0,1.0,0.0,0.0,0.0,Security staff,2.0,1,"This application was approved due to,Marital S..."
3,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52 years old,3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,1,"This application was approved due to Email,Num..."
4,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52 years old,3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,1,"This application was denied due to,Email 1.0,H..."


In [12]:
# Convert "Employment Duration (Days)" to years in-place
df["Employment Duration (Days)"] = df["Employment Duration (Days)"].apply(lambda x: round(x / 365.0, 2) if not pd.isna(x) else np.nan)
df.rename(columns={"Age (Days)": "Age (Years)", "Employment Duration (Days)": "Employment Duration (Years)"}, inplace=True)
df.head()

,Gender,Car Ownership,Property Ownership,Number of Children,Annual Income,Income Category,Education Level,Marital Status,Housing Type,Age (Years),Employment Duration (Years),Mobile Phone,Work Phone,Phone,Email,Occupation,Family Size,Approved,Reason
0,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32 years old,12.44,1.0,1.0,0.0,0.0,NaN,2.0,1,This application was approved due to Employmen...
1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32 years old,12.44,1.0,1.0,0.0,0.0,NaN,2.0,1,This application was approved due to Employmen...
2,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,58 years old,3.11,1.0,0.0,0.0,0.0,Security staff,2.0,1,"This application was approved due to,Marital S..."
3,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52 years old,8.36,1.0,0.0,1.0,1.0,Sales staff,1.0,1,"This application was approved due to Email,Num..."
4,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52 years old,8.36,1.0,0.0,1.0,1.0,Sales staff,1.0,1,"This application was denied due to,Email 1.0,H..."


In [13]:
# Step 2: Update the "Reason" column to match the new labels
def update_reason(reason_text):
    reason_text = reason_text.replace("Age (Days)", "Age (Years)")
    reason_text = reason_text.replace("Employment Duration (Days)", "Employment Duration (Years)")
    return reason_text

df["Reason"] = df["Reason"].apply(update_reason)

In [14]:
# Display the first value in the 'Reason' column
print(df['Reason'].iloc[0])

This application was approved due to Employment Duration (Years),Housing Type Rented apartment,Marital Status Civil marriage,Work Phone 1.0,Number of Children 0,Family Size 2.0,Annual Income 427500.0.


In [15]:
def generate_plain_input(row):
    # Generate the reasoning part of the input text
    reasoning_parts = [
        f"Gender: {'Male' if row['Gender'] == 'M' else 'Female'}" if pd.notna(row.get('Gender')) else None,
        f"Age: {row['Age (Years)']} years old" if pd.notna(row.get('Age (Years)')) else None,
        f"Car Ownership: {'Yes' if row['Car Ownership'] == 'Y' else 'No'}" if pd.notna(row.get('Car Ownership')) else None,
        f"Property Ownership: {'Yes' if row['Property Ownership'] == 'Y' else 'No'}" if pd.notna(row.get('Property Ownership')) else None,
        f"Number of Children: {int(row['Number of Children'])}" if pd.notna(row.get('Number of Children')) else None,
        f"Annual Income: {row['Annual Income']}" if pd.notna(row.get('Annual Income')) else None,
        f"Income Category: {row['Income Category']}" if pd.notna(row.get('Income Category')) else None,
        f"Education Level: {row['Education Level']}" if pd.notna(row.get('Education Level')) else None,
        f"Marital Status: {row['Marital Status']}" if pd.notna(row.get('Marital Status')) else None,
        f"Housing Type: {row['Housing Type']}" if pd.notna(row.get('Housing Type')) else None,
        f"Employment Duration: {row['Employment Duration (Years)']} years" if pd.notna(row.get('Employment Duration (Years)')) else None,
        f"Mobile Phone: {'Yes' if row['Mobile Phone'] == 1 else 'No'}" if pd.notna(row.get('Mobile Phone')) else None,
        f"Work Phone: {'Yes' if row['Work Phone'] == 1 else 'No'}" if pd.notna(row.get('Work Phone')) else None,
        f"Email: {'Yes' if row['Email'] == 1 else 'No'}" if pd.notna(row.get('Email')) else None,
        f"Family Size: {row['Family Size']}" if pd.notna(row.get('Family Size')) else None,
    ]
    input_text = ". ".join([part for part in reasoning_parts if part is not None])
    return input_text

In [16]:
# Apply the function to create the "text" column
df['text'] = df.apply(generate_plain_input, axis=1)

# Rename the "Reason" column to "label"
df.rename(columns={"Reason": "label"}, inplace=True)

# Create a final DataFrame with the "text" and "label" columns
final_df = df[['text', 'label']]

In [17]:
print(final_df['text'].iloc[0])
print(final_df['label'].iloc[0])

# Display the dataframe
final_df.head()

Gender: Male. Age: 32 years old years old. Car Ownership: Yes. Property Ownership: Yes. Number of Children: 0. Annual Income: 427500.0. Income Category: Working. Education Level: Higher education. Marital Status: Civil marriage. Housing Type: Rented apartment. Employment Duration: 12.44 years. Mobile Phone: Yes. Work Phone: Yes. Email: No. Family Size: 2.0
This application was approved due to Employment Duration (Years),Housing Type Rented apartment,Marital Status Civil marriage,Work Phone 1.0,Number of Children 0,Family Size 2.0,Annual Income 427500.0.


,text,label
0,Gender: Male. Age: 32 years old years old. Car...,This application was approved due to Employmen...
1,Gender: Male. Age: 32 years old years old. Car...,This application was approved due to Employmen...
2,Gender: Male. Age: 58 years old years old. Car...,"This application was approved due to,Marital S..."
3,Gender: Female. Age: 52 years old years old. C...,"This application was approved due to Email,Num..."
4,Gender: Female. Age: 52 years old years old. C...,"This application was denied due to,Email 1.0,H..."


In [18]:
# instruct_base_model = "meta-llama/Llama-3.1-8B-Instruct"
instruct_new_model = "/opt/notebooks/Chatbot-Credit-Card/backend/models/llama-3.1-8b-Instruct-CC/"
instruct_base_model = "/opt/notebooks/Chatbot-Credit-Card/backend/models/llama-3.1-8b-Instruct/"

epochs=1

In [19]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [20]:
# Convert DataFrames to HuggingFace datasets
final_dataset = Dataset.from_pandas(final_df)
# random_dataset = Dataset.from_pandas(random_df)

# Split datasets into train and eval
final_split = final_dataset.train_test_split(test_size=0.2)
# random_split = random_dataset.train_test_split(test_size=0.2)

final_train_dataset = final_split['train']
final_eval_dataset = final_split['test']

# random_train_dataset = random_split['train']
# random_eval_dataset = random_split['test']

In [21]:
# Custom Callback to save model every 5 epochs
class SaveEveryNEpochsCallback(TrainerCallback):
    def __init__(self, save_every_n_epochs, output_dir):
        self.save_every_n_epochs = save_every_n_epochs
        self.output_dir = output_dir

    def on_epoch_end(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        if state.epoch % self.save_every_n_epochs == 0:
            model_save_path = os.path.join(self.output_dir, f"checkpoint-{int(state.epoch)}-epochs")
            kwargs["model"].save_pretrained(model_save_path)
            kwargs["tokenizer"].save_pretrained(model_save_path)
            print(f"Model saved at {model_save_path}")

In [22]:
run = wandb.init(
    project='Fine-tune Llama 3 on CC Dataset', 
    job_type="training", 
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dnicho26 (dnicho26-university-of-north-carolina-at-charlotte). Use `wandb login --relogin` to force relogin


In [23]:
def get_training_arguments(output_dir, num_epochs):
    return TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=16,  
        optim="adamw_bnb_8bit",  # Optimized optimizer
        num_train_epochs=num_epochs,
        eval_strategy="steps",
        eval_steps=100,  # Less frequent evaluation
        logging_steps=100,  # Align with eval_steps
        warmup_steps=10,
        logging_strategy="steps",
        learning_rate=2e-4,
        fp16=True,  # Mixed precision
        dataloader_num_workers=8,  # Speed up data loading
        group_by_length=True,
        load_best_model_at_end=True,  # Ensure this is set
        report_to="wandb"
    )

In [24]:
def log_labels_and_inputs(batch, predictions, step):
    """
    Log inputs, labels, and predictions to wandb.
    """
    inputs = tokenizer.batch_decode(batch["input_ids"], skip_special_tokens=True)
    labels = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)
    preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    for input_text, label_text, pred_text in zip(inputs, labels, preds):
        wandb.log({
            "input": input_text,
            "label": label_text,
            "prediction": pred_text,
            "step": step,
        })


In [25]:
# Training and evaluation function
def train_and_save_model(model, tokenizer, train_dataset, eval_dataset, peft_config, new_model_path, output_suffix, num_epochs=epochs, save_every_n_epochs=5):
    full_training_args = get_training_arguments(new_model_path + output_suffix, num_epochs)
    model.train()
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        peft_config=peft_config,
        max_seq_length=512,
        dataset_text_field="text",
        tokenizer=tokenizer,
        args=full_training_args,
        packing=False,
        # data_collator=data_collator,  # Add this

    )

    trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=5))
    trainer.add_callback(SaveEveryNEpochsCallback(save_every_n_epochs=save_every_n_epochs, output_dir=new_model_path + output_suffix))

    # Custom evaluation logging
    def evaluation_logging(trainer, eval_dataset):
        predictions = trainer.predict(eval_dataset)
        step = trainer.state.global_step
        for batch in eval_dataset: 
            log_labels_and_inputs(batch, predictions, step)

    # Train the model
    trainer.train()

    # Log evaluation after training
    evaluation_logging(trainer, eval_dataset)

    # Save the final trained model
    model.save_pretrained(new_model_path + output_suffix + "-final")
    tokenizer.save_pretrained(new_model_path + output_suffix + "-final")
    print(f"Final model saved at {new_model_path + output_suffix + '-final'}")

In [26]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [27]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

def load_model_and_tokenizer(base_model_path, device_map="auto", max_memory=None, attn_implementation="default"):
    model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        quantization_config=bnb_config,
        device_map=device_map,
        max_memory=max_memory,
        attn_implementation=attn_implementation
    )
    tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)
    return model, tokenizer

# modules = find_all_linear_names(model)
# print(modules)
# Define LoRA config once since both models use it
def create_lora_config(model):
    # modules = find_all_linear_names(model)
    return LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",]
    )


In [28]:
def test_model(model, tokenizer, input_text):
    # Construct the prompt using the same format as in training
    prompt = f"Input:\n{input_text}\n\nLabel:\n"

    # Tokenize and generate response
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
    outputs = model.generate(
        **inputs,
        attention_mask=inputs.get('attention_mask'),
        max_new_tokens=150,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,  # Ensure decoding stops correctly
        forced_bos_token_id=tokenizer.bos_token_id,  # Force beginning of sequence token
        temperature=0.8,
        top_p=0.9,
        repetition_penalty=1.2,
    )

    # Decode response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the generated Label section
    response_content = response.split("Label:\n")[-1].strip()
    return response_content

In [29]:
# Define test prompts
# Create an input text
input_text = (
    "Gender: Male. "
    "Age: 32 years old. "
    "Car Ownership: Yes. "
    "Property Ownership: Yes. "
    "Number of Children: 0. "
    "Annual Income: 427500.0. "
    "Income Category: Working. "
    "Education Level: Higher education. "
    "Marital Status: Civil marriage. "
    "Housing Type: Rented apartment. "
    "Employment Duration: 12.44 years. "
    "Mobile Phone: Yes. "
    "Work Phone: Yes. "
    "Email: No. "
    "Family Size: 2.0."
)

In [30]:
# Load Instruct Model
print("Loading Instruct Model...")
instruct_model, instruct_tokenizer = load_model_and_tokenizer(
    base_model_path=instruct_base_model,
    device_map="auto",
    max_memory={0: "16GiB", "cpu": "30GiB"},
    attn_implementation=attn_implementation
)
instruct_tokenizer.pad_token = instruct_tokenizer.eos_token
instruct_model = prepare_model_for_kbit_training(instruct_model)

# Assert tokenizer and model embedding size match
assert instruct_model.get_input_embeddings().weight.size(0) == len(instruct_tokenizer), (
    f"Model embedding size after peft ({instruct_model.get_input_embeddings().weight.size(0)}) "
    f"does not match tokenizer vocabulary size ({len(instruct_tokenizer)})."
)

Loading Instruct Model...


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [31]:
# from transformers import PreTrainedTokenizer, DataCollatorWithPadding

# def tokenize_inputs_and_labels(examples):
#     # Tokenize the text input
#     input_tokens = instruct_tokenizer(
#         examples["text"],  # Access the "text" field
#         truncation=True,
#         padding="max_length",
#         max_length=512,
#     )
#     # Tokenize the label text
#     label_tokens = instruct_tokenizer(
#         examples["label"],  # Access the "label" field
#         truncation=True,
#         padding="max_length",
#         max_length=512,
#     )
#     # Add tokenized labels as a new field
#     input_tokens["labels"] = label_tokens["input_ids"]
#     return input_tokens

# # Apply the tokenizer to the dataset
# tokenized_dataset = final_dataset.map(tokenize_inputs_and_labels, batched=True)

# # Remove unnecessary columns (keep only the processed tokens)
# tokenized_dataset = tokenized_dataset.remove_columns(["text", "label"])

# # Split the tokenized dataset into train and test sets
# final_split = tokenized_dataset.train_test_split(test_size=0.2)

# final_train_dataset = final_split["train"]
# final_eval_dataset = final_split["test"]

# # Data collator to handle dynamic padding
# data_collator = DataCollatorWithPadding(tokenizer=instruct_tokenizer, padding=True)

# # Inspect some tokenized labels for verification
# print(final_train_dataset[0]["labels"][:10])  # Print the first 10 label tokens

In [ ]:
peft_config = create_lora_config(instruct_model)

# Apply PEFT to the model
instruct_model = get_peft_model(instruct_model, peft_config)

instruct_model.gradient_checkpointing_enable()

# Set model to training mode
instruct_model.train()

# Train Instruct Model on random Dataset
print("Training Instruct Model on Final Dataset...")
train_and_save_model(
    model=instruct_model,
    new_model_path=instruct_new_model,
    tokenizer=instruct_tokenizer,
    train_dataset=final_train_dataset,
    eval_dataset=final_eval_dataset,
    peft_config=peft_config,
    output_suffix="instruct",
    num_epochs=epochs
)

Training Instruct Model on Final Dataset...


Map:   0%|          | 0/6759 [00:00<?, ? examples/s]

Map:   0%|          | 0/1690 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss


In [ ]:
print("Testing instruct model...")
instruct_response = test_model(base_model, base_tokenizer, messages)
print("Instruct Model Response:")
print(instruct_response)

In [ ]:
# Close WandB session
wandb.finish()